In [1]:
import json
import yfinance as yf
import numpy as np
from scipy.stats import gaussian_kde

# Step 1: Retrieve data for both stocks
ticker1 = 'AAPL'
ticker2 = 'MSFT'
start_date = '2000-01-01'
end_date = '2024-01-01'

# Download data for each ticker
data1 = yf.download(ticker1, start=start_date, end=end_date, interval='1wk')
data1['Return'] = data1['Adj Close'].pct_change().dropna()

data2 = yf.download(ticker2, start=start_date, end=end_date, interval='1wk')
data2['Return'] = data2['Adj Close'].pct_change().dropna()

# Extract returns
returns_stock1 = data1['Return'].dropna()
returns_stock2 = data2['Return'].dropna()

# Step 2: KDE for each stock's returns
kde1 = gaussian_kde(returns_stock1)
kde2 = gaussian_kde(returns_stock2)

# Create x-axis range for KDE
x_grid = np.linspace(min(returns_stock1.min(), returns_stock2.min()),
                     max(returns_stock1.max(), returns_stock2.max()), 1000)

pdf_stock1 = kde1(x_grid)
pdf_stock2 = kde2(x_grid)

# Step 3: CDF Calculation
cdf_stock1 = np.cumsum(pdf_stock1) * (x_grid[1] - x_grid[0])
cdf_stock2 = np.cumsum(pdf_stock2) * (x_grid[1] - x_grid[0])

# Step 4: ICDF Calculation (integrated CDF)
icdf_stock1 = np.cumsum(cdf_stock1) * (x_grid[1] - x_grid[0])
icdf_stock2 = np.cumsum(cdf_stock2) * (x_grid[1] - x_grid[0])

# Step 5: Save data as JSON for use in Svelte
output_data = {
    "x_grid": x_grid.tolist(),
    "pdf_stock1": pdf_stock1.tolist(),
    "pdf_stock2": pdf_stock2.tolist(),
    "cdf_stock1": cdf_stock1.tolist(),
    "cdf_stock2": cdf_stock2.tolist(),
    "icdf_stock1": icdf_stock1.tolist(),
    "icdf_stock2": icdf_stock2.tolist(),
}

# Write to JSON file
with open('stock_analysis_data.json', 'w') as f:
    json.dump(output_data, f)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
